# 제 1고지 미분 자동 계산

###### 1단계 상자로서의 변수
###### 2단계 변수를 낳는 함수
###### 3단계 함수 연결
###### 4단계 수치 미분
###### 5단계 역전파 이론
###### 6단계 수동 역전파 이론
###### 7단계 역전파 자동화
###### 8단계 재귀에서 반복문으로
###### 9단계 함수를 더 편리하게
###### 10단계 테스트

In [7]:
import numpy as np
import matplotlib as plt

## step 1 상자로서의 변수

In [2]:
# Variable 클래스 구현

class Variable :
    def __init__(self, data) :
        self.data = data

In [3]:


data = np.array(1.0)
x = Variable(data)

In [4]:
print(x.data)

1.0


In [5]:
x.data = np.array(2.0)

In [6]:
print(x.data)

2.0


## step2 변수를 낳는 함수

In [8]:
# Function 클래스 구현
class Function :
    def __call__(self, input) :
        x = input.data
        y = x**2
        output = Variable(y)
        return output

In [9]:
x = Variable(np.array(10))
f = Function()
y = f(x)

In [12]:
print(y)

In [13]:
print(y.data)

100


In [3]:
# real implementation

class Function :
    
    def __call__(self, input) :
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output
        
    def forward(self, x) :
        raise NotImplementedError()

In [4]:
# square implementation

class Square(Function) :
    def forward(self, x) :
        return x ** 2

In [18]:
x = Variable(np.array(100))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
10000


## step 3 함수 연결

In [16]:
## exp 함수 구현

class Exp(Function) :
    def forward(self, x) :
        return np.exp(x)
    

In [20]:
A = Square()
B = Exp()
C = Square()

In [24]:
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


## step4 수치 미분

In [12]:
# 미분 개념
# 중앙 차분

def numerical_diff(f, x, eps = 1e-4) :
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2*eps)

In [13]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [14]:
# 합성함수의 미분

def f(x) :
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

In [17]:
x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


In [20]:
# 안 씀
# 근데 오차 커질 수 있다. 자릿수 누락과 함께
# 계산량이 너무 많다.



## step5 역전파 이론

In [21]:
# 손실함수인 y에 대한 각 파라미터의 그래디언트를 구하기 위해서임
# 순전파 & 역전파 
# 먼저 순전파하고 각 함수가 입력변수의 값 기억
# 그 값을 토대로 역전파 수행

# 통상값 (인퍼런스한 값) vs 미분값

## step6 수동 역전파

In [28]:
class Variable :
    def __init__(self, data) :
        self.data = data
        self.grad = None

In [29]:
# 미분을 계산하는 역전파
# forward 메소드 호출 시 건네받은 Variable 인스턴스 유지

class Function :
    def __call__(self, input) :
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        return output
    
    def forward(self, x) :
        raise NotImplementedError()
    
    def backward(self, gy) :
        raise NotImplementedError()

In [30]:
## Square 
class Square(Function) :
    def forward(self, x) :
        y = x ** 2 
        return y

    def backward(self, gy) :
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [32]:
## Exp
class Exp(Function) :
    def forward(self, x) :
        y = np.exp(x)
        return y

    def backward(self, gy) :
        x = self.input.data
        gx = np.exp(x) * gy
        return gx
        

In [40]:
# forward

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [43]:
# backward
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)

print(x.grad)

3.297442541400256
